### Import Packages

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import random
from collections import namedtuple, deque

from unityagents import UnityEnvironment

import matplotlib.pyplot as plt

from bagent import Agent

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
env = UnityEnvironment(file_name='C:\\Users\\austi\\Desktop\\Reacher_Arm_Control\\Reacher_Windows_x86_64\\Reacher.exe')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Train Agent

In [5]:
num_agents = len(env_info.agents)
max_t = 1e5
n_episodes = 300
LR = 3e-4          # learning rate
EPSILON = 1e-5      # Adam epsilon
ROLLOUT_LENGTH = 1024

state_size = env_info.vector_observations.shape[1]
hidden_size = 512
action_size = brain.vector_action_space_size


agent = Agent( num_agents, state_size, action_size)

# TO CONTINUE TRAINING:
# agent.model.load_state_dict(torch.load('ppo_checkpoint.pth'))


def test_agent(env, brain_name):
    env_info = env.reset(train_mode = True)[brain_name]
    states = env_info.vector_observations
    scores = np.zeros(num_agents)
    while True:
        actions, _, _= agent.act(states)
        env_info = env.step(actions.cpu().detach().numpy())[brain_name]
        next_states = env_info.vector_observations
        rewards = env_info.rewards
        dones = env_info.local_done
        scores += env_info.rewards
        states = next_states
        if np.any(dones):
            break
    return np.mean(scores)

if True:
    env.info = env.reset(train_mode = True)[brain_name]
    scores = []
    scores_window = deque(maxlen=100)
    aus_scores = deque(maxlen=100)
        
    for i_episode in range(1, n_episodes+1):
        # Each iteration, N parallel actors collect T time steps of data
        
        # AGENT: def step(self, rollout, num_agents):
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations  
        rollout = []
        agents_rewards = []
        episode_rewards = []
        agent_scores = np.zeros(num_agents)

        for _ in range(ROLLOUT_LENGTH):
            actions, log_probs, values = agent.act(states)
            env_info = env.step(actions.cpu().detach().numpy())[brain_name]
            next_states = env_info.vector_observations
            rewards = env_info.rewards
            dones = np.array([1 if d else 0 for d in env_info.local_done])
            agents_rewards += rewards 
            agent_scores += rewards
            
            # If an agent is done, append its reward and reset it to 0
            for j, done in enumerate(dones):
                if dones[j]:
                    episode_rewards.append(agents_rewards[j])
                    agents_rewards[j] = 0
                    
            # append tuple ( s, a, p(a|s), r, dones, V(s) )
            rollout.append([states, actions.detach(), log_probs.detach(), rewards, 1 - dones, values.detach()])
            states = next_states
            if np.any(dones):                                  # exit loop if episode finished
                break

        # Value for next step
        pending_values = agent.model(states)[-1].detach()
        rollout.append([states, None, None, None, None, pending_values])
        
        agent.step(rollout, num_agents)
        
        test_mean_reward = test_agent(env, brain_name)
        scores.append(test_mean_reward)
        scores_window.append(test_mean_reward)
        aus_scores.append(np.mean(agent_scores))

        if np.mean(scores_window) > 30.0:
            torch.save(agent.model.state_dict(), f"ppo_checkpoint.pth")
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, np.mean(all_scores_window)))
            break       
        
        print('Episode {}, Total score this episode: {}, Last {} average: {}'.format(i_episode, test_mean_reward, min(i_episode, 100), np.mean(scores_window)) )
        #print('Episode {}, Total score this episode: {}, Last {} average: {}'.format(i_episode, test_mean_reward, min(i_episode, 100), np.mean(scores_window)) )

c:\users\austi\miniconda3\envs\drlnd2\lib\site-packages\torch\nn\functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Entropy:	 tensor(1.4189, grad_fn=<MeanBackward1>)
Sur:	 tensor([[ 0.3734],
        [-1.6101],
        [ 0.1887],
        [ 1.4932],
        [ 0.6464],
        [-0.8113],
        [-1.3501],
        [ 0.8822],
        [-0.2378],
        [-1.9010],
        [ 0.4505],
        [-2.1335],
        [-1.2861],
        [ 1.0692],
        [ 1.2133],
        [ 1.2454],
        [ 0.0949],
        [ 2.3273],
        [ 1.1033],
        [-0.5521],
        [-1.0084],
        [ 0.4866],
        [ 0.1217],
        [-0.6409],
        [-0.0178],
        [-0.5206],
        [ 0.2715],
        [ 0.2776],
        [-0.5316],
        [-0.2889],
        [ 0.1099],
        [ 0.1252]], grad_fn=<MulBackward0>)
Clipped:	 tensor([[ 0.3734],
        [-1.6101],
        [ 0.1887],
        [ 1.4932],
        [ 0.6464],
        [-0.8113],
        [-1.3501],
        [ 0.8822],
        [-0.2378],
        [-1.9010],
        [ 0.4505],
        [-2.1335],
        [-1.2861],
        [ 1.0692],
        [ 1.2133],
        [ 1.2454

Clipped:	 tensor([[-0.3858],
        [ 0.1165],
        [-0.1713],
        [ 0.8454],
        [ 0.4422],
        [ 0.4432],
        [-0.7424],
        [-2.1430],
        [ 0.3558],
        [-0.8080],
        [ 0.8446],
        [ 0.1474],
        [-1.9395],
        [-1.0628],
        [-1.5742],
        [ 0.7923],
        [ 0.3404],
        [-0.1903],
        [ 1.0778],
        [ 0.3780],
        [-1.0261],
        [ 0.6188],
        [ 1.2638],
        [-0.4130],
        [ 0.5819],
        [ 1.3097],
        [ 1.1282],
        [-0.0504],
        [ 0.0165],
        [-0.1333],
        [ 0.9943],
        [ 0.5902]], grad_fn=<MulBackward0>)
Policy Loss:  tensor([-0.0457], grad_fn=<SubBackward0>)
Value Loss:  tensor(0.0095, grad_fn=<MeanBackward1>)
Entropy:	 tensor(1.4185, grad_fn=<MeanBackward1>)
Sur:	 tensor([[-0.7787],
        [ 0.4868],
        [ 0.2172],
        [-1.8618],
        [ 1.0913],
        [-0.7588],
        [ 0.3974],
        [ 0.6100],
        [ 0.2072],
        [-1.2507],
  

Entropy:	 tensor(1.4186, grad_fn=<MeanBackward1>)
Sur:	 tensor([[-0.5128],
        [ 1.0792],
        [ 0.8816],
        [ 1.2489],
        [ 0.9927],
        [-1.1450],
        [ 0.0910],
        [ 0.6735],
        [-0.2980],
        [ 0.6773],
        [ 0.6966],
        [ 0.8233],
        [ 0.4579],
        [ 0.0785],
        [-0.9796],
        [ 0.2741],
        [-1.8115],
        [-1.6707],
        [ 0.9554],
        [-0.8847],
        [ 0.2269],
        [ 1.4010],
        [ 0.6403],
        [ 2.0344],
        [-0.1457],
        [ 0.0992],
        [-0.8801],
        [ 0.1255],
        [ 0.3221],
        [-0.8758],
        [ 1.3317],
        [-0.0245]], grad_fn=<MulBackward0>)
Clipped:	 tensor([[-0.5128],
        [ 1.0792],
        [ 0.8882],
        [ 1.2489],
        [ 0.9927],
        [-1.0785],
        [ 0.0910],
        [ 0.8485],
        [-0.2980],
        [ 0.6773],
        [ 0.6966],
        [ 0.8233],
        [ 0.4579],
        [ 0.0916],
        [-0.9796],
        [ 0.2741

Entropy:	 tensor(1.4185, grad_fn=<MeanBackward1>)
Sur:	 tensor([[ 0.2615],
        [ 0.1630],
        [-1.0581],
        [-0.9698],
        [ 0.4924],
        [ 0.0246],
        [-0.4961],
        [ 0.9623],
        [-0.1028],
        [ 0.2146],
        [-0.7864],
        [ 1.2345],
        [ 0.1205],
        [ 0.6143],
        [ 1.4706],
        [-1.5606],
        [-1.7828],
        [-0.4950],
        [ 0.9916],
        [ 1.1948],
        [ 0.7147],
        [ 0.1593],
        [ 1.3051],
        [ 0.3670],
        [-0.0351],
        [-2.4179],
        [ 0.9880],
        [-0.0387],
        [-0.9596],
        [-2.3403],
        [ 0.9357],
        [ 0.6885]], grad_fn=<MulBackward0>)
Clipped:	 tensor([[ 0.2615],
        [ 0.1350],
        [-1.0581],
        [-0.9655],
        [ 0.4924],
        [ 0.0246],
        [-0.4961],
        [ 0.9623],
        [-0.1028],
        [ 0.2333],
        [-0.7864],
        [ 1.2345],
        [ 0.2130],
        [ 0.6143],
        [ 1.1072],
        [-1.5606

Entropy:	 tensor(1.4187, grad_fn=<MeanBackward1>)
Sur:	 tensor([[ 0.3084],
        [ 1.1248],
        [ 0.1510],
        [-0.3673],
        [ 0.8229],
        [ 1.0149],
        [-0.1750],
        [ 1.4189],
        [-0.3161],
        [ 0.3917],
        [ 0.0554],
        [-1.4227],
        [-0.6514],
        [ 0.5567],
        [-0.1545],
        [-0.0543],
        [-0.0026],
        [ 1.0665],
        [ 0.7657],
        [ 0.1304],
        [-1.1015],
        [ 0.7786],
        [-0.4189],
        [-0.9203],
        [-0.1330],
        [-0.0320],
        [-1.8605],
        [ 1.5097],
        [-0.5859],
        [-0.5926],
        [ 0.3041],
        [ 0.0738]], grad_fn=<MulBackward0>)
Clipped:	 tensor([[ 0.3084],
        [ 1.1248],
        [ 0.2091],
        [-0.3673],
        [ 0.8229],
        [ 1.0149],
        [-0.1750],
        [ 1.4189],
        [-0.3161],
        [ 0.3917],
        [ 0.0554],
        [-1.4227],
        [-0.6514],
        [ 0.5567],
        [-0.1545],
        [-0.0543

Entropy:	 tensor(1.4190, grad_fn=<MeanBackward1>)
Sur:	 tensor([[ 0.0126],
        [-0.9136],
        [ 0.5613],
        [-1.5228],
        [-0.0607],
        [-1.5252],
        [ 0.9149],
        [ 0.1318],
        [-0.5277],
        [ 1.0724],
        [-1.5018],
        [ 1.0449],
        [ 0.7571],
        [ 1.4798],
        [-0.1586],
        [-0.0431],
        [ 1.5578],
        [ 0.2883],
        [ 2.3226],
        [-1.4705],
        [ 0.3711],
        [-1.6250],
        [-0.1693],
        [ 1.3799],
        [ 0.0169],
        [ 0.7240],
        [-0.1402],
        [-1.1793],
        [ 0.7137],
        [-3.0149],
        [ 0.1988],
        [ 0.0239]], grad_fn=<MulBackward0>)
Clipped:	 tensor([[ 0.0126],
        [-1.0499],
        [ 0.5613],
        [-1.5114],
        [-0.0653],
        [-1.5252],
        [ 0.9149],
        [ 0.1406],
        [-0.4843],
        [ 1.0724],
        [-1.2910],
        [ 1.1526],
        [ 0.7571],
        [ 1.4798],
        [-0.1586],
        [-0.0431

KeyboardInterrupt: 

### 4. Run Agent

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
for _ in range(5):
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    print (actions)
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

In [ ]:
env.close()